### Librarias

In [2]:
import sympy as sp
import numpy as np
from numpy.linalg import inv
from mpmath import *
x = sp.Symbol('x')

### Funções auxiliares

In [3]:
def matrizDosCoeficientes(objeto, grauDoNumerador, grauDoDenominador, prec):
    mp.dps = prec
    if type(objeto) == list:
        if grauDoNumerador + grauDoDenominador + 1 <= len(objeto):
            A = sp.Matrix( np.zeros(( grauDoDenominador, grauDoDenominador )) ) 
    
            # cálculo dos coeficientes da série de Taylor em x = 0
            for linha in range( 0, grauDoDenominador ):  
                for coluna in range(0, grauDoDenominador):
                    if ( grauDoDenominador - grauDoDenominador + coluna + 1 + linha < 0 ):
                        A[ linha, coluna ] = 0
                
                    elif ( grauDoDenominador - grauDoDenominador + coluna + 1 + linha == 0 ):
                        an = mpf(objeto[0]) 
                        A[ linha, coluna ] = sp.Float( an, prec )
                
                    else:
                        an = objeto[ grauDoDenominador - grauDoDenominador + coluna + 1 + linha ]
                        A[ linha, coluna] = sp.Float( an, prec )
            return(A)
        else:
            print('O número de coeficientes não é suficiente.')
            return
        
    else:
        mp.dps = prec
        x = sp.Symbol('x')
        # matriz nula grauDoDenominadorxgrauDoDenominador
    
        A = sp.Matrix( np.zeros(( grauDoDenominador, grauDoDenominador )) ) 
    
        # cálculo dos coeficientes da série de Taylor em x = 0
    
        for linha in range( 0, grauDoDenominador ):  
            for coluna in range(0, grauDoDenominador):
                if ( grauDoDenominador - grauDoDenominador + coluna + 1 + linha < 0 ):
                    A[ linha, coluna ] = 0
                
                elif ( grauDoDenominador - grauDoDenominador + coluna + 1 + linha == 0 ):
                    fn = sp.diff( objeto, x, 0)
                    fnsub = fn.subs( x, 0 )
                    an = sp.N(fnsub, prec) 
                    A[ linha, coluna ] = sp.Float( an, prec )
                
                else:
                    fn = sp.diff( objeto, x, grauDoDenominador - grauDoDenominador + coluna + 1 + linha )
                    fnsub = fn.subs( x, 0 )
                    factorialn = factorial( grauDoDenominador - grauDoDenominador + coluna + linha + 1 )
            
                    an = mpf( sp.N(fnsub, prec) / mpf(factorialn))
                    A[ linha, coluna] = sp.Float( an, prec )
                
        return ( A )

In [4]:
def matrizDosTermosIndependentes(objeto, grauDoNumerador, grauDoDenominador, prec):
    mp.dps = prec
    if type(objeto) == list:
        if grauDoNumerador + grauDoDenominador + 1 <= len(objeto):
            # vetor nulo 1xq
            c = sp.Matrix( np.zeros(( 1, grauDoDenominador )) ) 
            # cálculo dos coeficientes da série de Taylor em x = 0 
            for j in range( 0, grauDoDenominador ):
                an = objeto[ grauDoNumerador + j +1]   
                
                c[ 0, j ] = ( -1 ) * sp.Float( an, prec )
            return (c)
    
        else:
            print('O número de coeficientes não é suficiente.')
            return
    
    else:
        x = sp.Symbol('x')
        # vetor nulo 1xq
    
        c = sp.Matrix( np.zeros(( 1, grauDoDenominador )) ) 
    
    
        # cálculo dos coeficientes da série de Taylor em x = 0 
    
        for coluna in range( 0, grauDoDenominador ):
            derivada = sp.diff( objeto, x, grauDoNumerador + coluna + 1 )
            derivadaEmZero =  derivada.subs( x, 0 )
            
            nl =  factorial( grauDoNumerador + coluna + 1  )
            an = mpf( sp.N(derivadaEmZero, prec) / mpf( nl ))
            
            c[ 0, coluna ] = ( -1 ) * sp.Float( an, prec )
                
        return (c)

In [5]:

def coeficientesParaCalcularCoeficentesDoNumerador(objeto, grauDoNumerador, grauDoDenominador, prec):
    mp.dps = prec
    if type(objeto) == list:
        if grauDoNumerador + grauDoDenominador + 1 <= len(objeto):
            cA = sp.Matrix(np.zeros(( 1, grauDoNumerador + 1 )))
    
            # cálculo dos coeficientes sa série de Taylor em x = 0 para o cálculo dos a'is
    
            for i in range( 0, grauDoNumerador + 1 ):
                if ( grauDoNumerador - i < 0 ):
                    cA[i] = 0
                else:
                    an = objeto[ grauDoNumerador - i] 
                    cA[i] = sp.Float( an, prec )
            return( cA )  
        else:
            print('O número de coeficientes não é suficiente.')
            return
    else:
        x = sp.Symbol('x')
        # vetor nulo 1x(p+1)
    
        cA = sp.Matrix(np.zeros(( 1, grauDoNumerador + 1 )))
    
        # cálculo dos coeficientes sa série de Taylor em x = 0 para o cálculo dos a'is
    
        for i in range( 0, grauDoNumerador + 1 ):
            if ( grauDoNumerador - i < 0 ):
                cA[i] = 0
            else:
                fn = sp.diff( objeto, x, grauDoNumerador - i )
                fn = fn.subs( x, 0 )
                factorialn = factorial( grauDoNumerador - i )
                an = mpf(sp.N(fn, prec )/mpf(factorialn))
                cA[i] = sp.Float( an, prec )
            
        return( cA )

### Cálculo do Padé  \[ p / q \]

In [13]:
def padeDiretoPrecisaoFinita(objeto, grauDoNumerador, grauDoDenominador, prec):
    x = sp.Symbol('x')
    # Precisão para a função mpf(.)
    mp.dps = prec
    
    A = matrizDosCoeficientes( objeto, grauDoNumerador, grauDoDenominador, prec)
    
    # Se a matriz A é invertível
    if(A is None): return
    
    if (type(A) is str) : return(A)
    
    if(A == 0): return("A matriz dos coeficientes não é invertível ")
        
    if ( A.det() != 0 ):
        A1 = A.inv( "LU" )
        Bn = matrizDosTermosIndependentes( objeto, grauDoNumerador, grauDoDenominador, prec ) * A.inv( "LU" )
        
        Dx = sp.Matrix( np.zeros(( grauDoDenominador + 1, 1 )) )
         
        for linha in range( 0, grauDoDenominador + 1 ):
                Dx[linha] = x**(linha)
        
        bn = sp.Matrix( np.zeros(( 1, grauDoDenominador + 1 )) )
        bn[0] = 1
        
        for coluna in range( 1, grauDoDenominador + 1 ):
            bn[coluna] = Bn[-coluna]

        Denominador = sp.Function( 'Denominador' )
        Denominador = bn * Dx
        

        
        cn = sp.Matrix( np.zeros(( 1, grauDoNumerador + 1 )) )
        

        Nx = sp.Matrix( np.zeros(( grauDoNumerador + 1, 1 )) )
        
        for linha in range( 0, grauDoNumerador + 1 ):
                Nx[linha] = x**(linha)

        
        # Se o grauDoNumerador é menor do que o grauDoDenominador
        
        if ( grauDoNumerador < grauDoDenominador ):
            coluna = 1
            iteracoes = 0
            while( coluna <= grauDoNumerador + 1 and iteracoes < 10**9 ):
                An = sp.Matrix( coeficientesParaCalcularCoeficentesDoNumerador(\
                                                objeto, grauDoNumerador, grauDoDenominador, prec )[ -coluna: ])
                Bn = sp.Matrix( bn[ 0:coluna ] )
                cn[ coluna - 1] = sp.Transpose( An ) * Bn
                coluna += 1
                iteracoes += 1
                
            Numerador = sp.Function( 'Numerador' )
            Numerador = cn * Nx
            R = sp.Function('R')
            R = Numerador / Denominador
            Pade = sp.Function('Pade')
            Pade = R[0]
            return ( Pade )
        
        # Se o grauDoNumerador é maior do que o grauDoDenominador
        
        else:
            coluna = 1
            iteracoes = 0
            while( coluna <= grauDoDenominador + 1 and iteracoes < 10**9 ):
                    
                    An= sp.Matrix( coeficientesParaCalcularCoeficentesDoNumerador(\
                                                            objeto, grauDoNumerador, grauDoDenominador, prec )[ -coluna: ] )
                    Bn = sp.Matrix( bn[ 0:coluna ] )
                    cn[ coluna - 1 ]=sp.Transpose( An ) * Bn
                    coluna += 1
                    iteracoes +=1
            j = -1
            iteracoes = 0
            while ( coluna <= grauDoNumerador + 1 and iteracoes < 10**9):
                    An = sp.Matrix( coeficientesParaCalcularCoeficentesDoNumerador(\
                                                        objeto, grauDoNumerador, grauDoDenominador, prec)[ -coluna:j ] )
                    Bn = sp.Matrix( bn[ 0:] )
                    cn[ coluna - 1 ] = sp.Transpose( An ) * Bn
                    coluna += 1
                    j -= 1
                    iteracoes += 1
                    
            Numerador = sp.Function( 'Numerador' )
            Numerador = cn * Nx
            R = sp.Function('R')
            R = Numerador / Denominador
            Pade = sp.Function('Pade')
            Pade = R[0]
        return ( Pade )
    
    # Se a matriz não é invertível
    else: return("A matriz dos coeficientes não é invertível ") 